The basic algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

In [2]:
# Read the words from the file
with open('words_250000_train.txt', 'r') as file:
    word_list = file.read().splitlines()
longest_word = max(word_list,key=len)
print("The longest word is:", longest_word)
print(len(longest_word))

The longest word is: cyclotrimethylenetrinitramine
29


### Thinking Process
I could think of 2 approaches for this problem, one was using reinforcement learning to learn using a given environment via some RL algorithm and the second was to use imitation learning (akin to supervised learning).
We first analyze the first approach.

#### Approach 1
I created a custom environment to play hangman (as written below) and trained the PPO algorithm with LSTM support. 
To check the feasibility of the approach, i trained the algorithm for 10000 timesteps while keeping the word to be guessed same (rabbit).

In [3]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from tqdm.notebook import tqdm

class HangmanEnv1(gym.Env):
    metadata = {"render_modes": ["human"], "render_fps": 30}
    def __init__(self, word_list,render_mode="human",rword=True):
        super(HangmanEnv1, self).__init__()
        self.word_list = word_list
        self.word = None
        self.obscured_word = None
        self.guessed_letters = set()
        self.tries_remaining = None
        self.rword = rword
        self.action_space = spaces.Discrete(26)  # Guessing a letter
        num_values = np.array([28 for _ in range(29)] + [2 for _ in range(26)] + [7])
        self.observation_space = spaces.MultiDiscrete(num_values)

    def reset(self,seed=None,options=None):
        if self.rword:
            self.word = np.random.choice(self.word_list)
        else:
            self.word = "rabbit"
        self.obscured_word = ['_' for _ in self.word]
        self.guessed_letters = set()
        self.tries_remaining = 6
        return self._get_observation(),{}

    def step(self, action):
        guess = chr(action + ord('a'))
        if guess not in self.guessed_letters:
            self.guessed_letters.add(guess)
        else:
            return self._get_observation(),-3,False,False,{}
        
        if guess in self.word:
            reward = 0
            for i in range(len(self.word)):
                if(self.word[i] == guess):
                    self.obscured_word[i] = guess
            if '_' not in self.obscured_word:
                return self._get_observation(),5,True,False,{}
            else:
                return self._get_observation(),2,False,False,{}
        else:
            self.tries_remaining -= 1
            if self.tries_remaining == 0:
                return self._get_observation(),-2,True,False,{}
            else:
                return self._get_observation(),-1,False,False,{}

    def _get_observation(self):
        return np.array([ord(c) - ord('a') if c != '_' else 26 for c in self.obscured_word]+[27 for _ in range(29-len(self.obscured_word))]
                +[1 if chr(c + ord('a')) in self.guessed_letters else 0 for c in range(26)]+
                [self.tries_remaining])
    def render(self):
        x = 10
    
def accuracy_fn(model,env,verbose=False):
    success = 0
    trr = 0
    progbar = tqdm(range(10000))
    for e in progbar:
        state,_ = env.reset()
        done = False
        tr = 0
        ep_len = 0
        while not done:
            action,_ = model.predict(state)
            next_state, reward, done,oo,bhoo = env.step(action)
            state = next_state
            tr += reward
            ep_len += 1
            if(reward > 0 and done):
                success+=1
            if verbose:
                print(env.word,env.guessed_letters)
        trr += tr/ep_len
    return success/100,trr

2024-06-29 19:34:24.287847: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-29 19:34:24.571390: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-29 19:34:25.296807: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
env = HangmanEnv1(word_list,rword=False)
model = PPO("MlpPolicy",env,verbose=1)
model.learn(total_timesteps=10000)
model.save("ppo_rabbit")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


KeyboardInterrupt: 

In [65]:
model_rabbit = PPO.load("ppo_rabbit",env)
percentage,mean_reward = accuracy_fn(model,env)
print("Success rate:",percentage)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


  0%|          | 0/10000 [00:00<?, ?it/s]

Success rate: 6.07


As you can see by means of the above cell the performance of the model is not admissible and is even lesser than the benchmark. So we conclude that this approach is not feasible.

In [9]:
# Create the Hangman environment
env1 = HangmanEnv1(word_list)
# Train the agent using PPO
model = PPO("MlpPolicy", env1,verbose=1)
model.learn(total_timesteps=5000000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 10.3     |
|    ep_rew_mean     | -7.58    |
| time/              |          |
|    fps             | 198      |
|    iterations      | 1        |
|    time_elapsed    | 10       |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 10.9         |
|    ep_rew_mean          | -7.66        |
| time/                   |              |
|    fps                  | 170          |
|    iterations           | 2            |
|    time_elapsed         | 24           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0133686215 |
|    clip_fraction        | 0.127        |
|    clip_range           | 0.2          |
|    en

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 9.71        |
|    ep_rew_mean          | -4.23       |
| time/                   |             |
|    fps                  | 145         |
|    iterations           | 11          |
|    time_elapsed         | 154         |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.015414957 |
|    clip_fraction        | 0.186       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.1        |
|    explained_variance   | 0.153       |
|    learning_rate        | 0.0003      |
|    loss                 | 1.95        |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.0525     |
|    value_loss           | 6.86        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 9.99  

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 10.9        |
|    ep_rew_mean          | -1.97       |
| time/                   |             |
|    fps                  | 147         |
|    iterations           | 21          |
|    time_elapsed         | 291         |
|    total_timesteps      | 43008       |
| train/                  |             |
|    approx_kl            | 0.023664542 |
|    clip_fraction        | 0.289       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.79       |
|    explained_variance   | 0.104       |
|    learning_rate        | 0.0003      |
|    loss                 | 1.97        |
|    n_updates            | 200         |
|    policy_gradient_loss | -0.068      |
|    value_loss           | 4.99        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 10.3  

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 11          |
|    ep_rew_mean          | 0.03        |
| time/                   |             |
|    fps                  | 147         |
|    iterations           | 31          |
|    time_elapsed         | 431         |
|    total_timesteps      | 63488       |
| train/                  |             |
|    approx_kl            | 0.027173044 |
|    clip_fraction        | 0.295       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.42       |
|    explained_variance   | 0.249       |
|    learning_rate        | 0.0003      |
|    loss                 | 1.93        |
|    n_updates            | 300         |
|    policy_gradient_loss | -0.0638     |
|    value_loss           | 4.49        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 11.1  

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 11.4        |
|    ep_rew_mean          | 0.95        |
| time/                   |             |
|    fps                  | 147         |
|    iterations           | 41          |
|    time_elapsed         | 569         |
|    total_timesteps      | 83968       |
| train/                  |             |
|    approx_kl            | 0.025857233 |
|    clip_fraction        | 0.268       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.17       |
|    explained_variance   | 0.308       |
|    learning_rate        | 0.0003      |
|    loss                 | 1.8         |
|    n_updates            | 400         |
|    policy_gradient_loss | -0.0617     |
|    value_loss           | 5.21        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 11.2  

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 11.4        |
|    ep_rew_mean          | 2.03        |
| time/                   |             |
|    fps                  | 297         |
|    iterations           | 2402        |
|    time_elapsed         | 16547       |
|    total_timesteps      | 4919296     |
| train/                  |             |
|    approx_kl            | 0.061530545 |
|    clip_fraction        | 0.21        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.428      |
|    explained_variance   | 0.552       |
|    learning_rate        | 0.0003      |
|    loss                 | 2.42        |
|    n_updates            | 24010       |
|    policy_gradient_loss | -0.0332     |
|    value_loss           | 3.72        |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 11      

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 12.1        |
|    ep_rew_mean          | 1.15        |
| time/                   |             |
|    fps                  | 296         |
|    iterations           | 2412        |
|    time_elapsed         | 16632       |
|    total_timesteps      | 4939776     |
| train/                  |             |
|    approx_kl            | 0.110925585 |
|    clip_fraction        | 0.134       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.453      |
|    explained_variance   | 0.169       |
|    learning_rate        | 0.0003      |
|    loss                 | 2.66        |
|    n_updates            | 24110       |
|    policy_gradient_loss | -0.0372     |
|    value_loss           | 18.8        |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 11.4    

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 11.6        |
|    ep_rew_mean          | 4.11        |
| time/                   |             |
|    fps                  | 296         |
|    iterations           | 2422        |
|    time_elapsed         | 16718       |
|    total_timesteps      | 4960256     |
| train/                  |             |
|    approx_kl            | 0.059543915 |
|    clip_fraction        | 0.201       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.447      |
|    explained_variance   | 0.124       |
|    learning_rate        | 0.0003      |
|    loss                 | 36.1        |
|    n_updates            | 24210       |
|    policy_gradient_loss | -0.057      |
|    value_loss           | 97.7        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 11.8  

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 11.6       |
|    ep_rew_mean          | 2.8        |
| time/                   |            |
|    fps                  | 296        |
|    iterations           | 2432       |
|    time_elapsed         | 16798      |
|    total_timesteps      | 4980736    |
| train/                  |            |
|    approx_kl            | 0.06294669 |
|    clip_fraction        | 0.213      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.444     |
|    explained_variance   | 0.634      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.941      |
|    n_updates            | 24310      |
|    policy_gradient_loss | -0.0347    |
|    value_loss           | 3.02       |
----------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 16.4     |
|    ep_rew_mean      

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 11.3        |
|    ep_rew_mean          | 3.43        |
| time/                   |             |
|    fps                  | 296         |
|    iterations           | 2442        |
|    time_elapsed         | 16877       |
|    total_timesteps      | 5001216     |
| train/                  |             |
|    approx_kl            | 0.028265957 |
|    clip_fraction        | 0.146       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.471      |
|    explained_variance   | 0.243       |
|    learning_rate        | 0.0003      |
|    loss                 | 10.2        |
|    n_updates            | 24410       |
|    policy_gradient_loss | -0.0314     |
|    value_loss           | 12.4        |
-----------------------------------------


In [17]:
model.save("ppo_5mil")

In [66]:
model_1 = PPO.load("ppo_5mil",env =env1)
percentage,mean_reward = accuracy_fn(model_1,env1)
print("Success rate:",percentage)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


  0%|          | 0/10000 [00:00<?, ?it/s]

Success rate: 0.0


I actually trained it for 5 million timesteps and random words too. It didn't have the desired accuracy on even the training set either.

#### Approach 2
We now think about using imitation learning to guess the word. I have used the following [github repository](https://github.com/YAPhoa/HangmanKeras.git) for reference. The changes i made to the code given there is make the network in pytorch instead of keras which decreased the training time by more than 10 times. I also made minor changes to the classes to decrease the training time even further

In [113]:
class Hangman:
    def __init__(self, words, max_tries = 6 ) :
        self.words = words
        self.max_tries = max_tries
        
    def reset(self) :
        self.tries_remaining = self.max_tries
        self.word = np.random.choice(self.words)
        self.obscurred_word = [27 for i in range(len(self.word))]
        self.guessed = set()
        self.correct_guesses = 0
        return self.obscurred_word
        
    def step(self, letter) :
        if letter not in self.guessed :
            self.guessed.add(letter)
        else :
            return self.obscurred_word, False, {}

        if letter in self.word :
            for i in range(len(self.word)) :
                if letter == self.word[i] :
                    self.obscurred_word[i] = ord(letter)-ord('a')+1
                    self.correct_guesses += 1
            if self.correct_guesses == len(self.word):
                return self.obscurred_word,True, {'ans' : self.word}
            else :                
                return self.obscurred_word,False,{}
        else :
            self.tries_remaining -= 1
            if self.tries_remaining == 0 :
                return self.obscurred_word,True, {'ans' : self.word}
            else :
                return self.obscurred_word,False,{}

In [114]:
from warnings import filterwarnings

filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tensorflow.keras.utils import pad_sequences

In [117]:
class StateEmbedding(nn.Module):
    def __init__(self, maxlen=29):
        super(StateEmbedding, self).__init__()
        self.embedding = nn.Embedding(30, 100, padding_idx=0)
        self.lstm1 = nn.LSTM(100, 100, num_layers=2,batch_first=True, bidirectional=True,dropout=0.2,bias=False)
        self.drp = nn.Dropout(0.2)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.dense = nn.Linear(200, 100)
    
    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm1(x)
        x = self.drp(x)
        x = x.permute(0, 2, 1)
        x = self.pool(x).squeeze(2)
        x = torch.tanh(self.dense(x))
        return x

class GuessedEmbedding(nn.Module):
    def __init__(self):
        super(GuessedEmbedding, self).__init__()
        self.dense1 = nn.Linear(26, 60)
        self.dense2 = nn.Linear(60, 60)
    
    def forward(self, x):
        x = torch.tanh(self.dense1(x))
        x = torch.tanh(self.dense2(x))
        return x

class Network(nn.Module):
    def __init__(self, maxlen=29):
        super(Network, self).__init__()
        self.state_embedding = StateEmbedding(maxlen)
        self.guessed_embedding = GuessedEmbedding()
        self.dense1 = nn.Linear(160, 100)
        self.dense2 = nn.Linear(100, 26)
        self.compile()
    
    def forward(self, state, guessed):
        state = torch.tensor(state, dtype=torch.long)
        guessed = torch.tensor(guessed, dtype=torch.float)
        state_emb = self.state_embedding(state)
        guessed_emb = self.guessed_embedding(guessed)
        x = torch.cat((state_emb, guessed_emb), dim=1)
        x = torch.tanh(self.dense1(x))
        x = self.dense2(x)
        return x

    def compile(self, lr=1e-3):
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.loss_fn = nn.CrossEntropyLoss()
    
    def save(self, path='model.pth'):
        torch.save(self.state_dict(), path)
    
    def load(self, path='model.pth'):
        self.load_state_dict(torch.load(path))

class Agent(object) :

    def __init__(self, model) :
        self.guessed = set()
        self.model = model

    def select_action(self,state) :            
        probs = self.get_probs(state)
        i = 1
        sorted_probs = probs.argsort()
        while chr(sorted_probs[-i]+ord('a')) in self.guessed :
            i+= 1
        idx = sorted_probs[-i]
        guess = chr(idx+ord('a'))
        if guess not in self.guessed :
            self.guessed.add(guess)
        return guess

class NNAgent(Agent) :
    def __init__(self, model) :
        super().__init__(model)
        self.episode_memory = []
        self.states_history = []

    def train_model(self):
        inp_1, inp_2, obj = zip(*self.states_history)
        inp_1 = np.vstack(list(inp_1)).astype(float)
        inp_2 = np.vstack(list(inp_2)).astype(float)
        obj = np.vstack(list(obj)).astype(float)
        self.model.train()
        inp_1 = torch.tensor(inp_1,dtype=torch.long)
        inp_2 = torch.tensor(inp_2,dtype=torch.float)
        obj = torch.tensor(obj,dtype=torch.float)
        y = self.model(inp_1,inp_2)
        loss = self.model.loss_fn(y,obj)
        self.model.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1)
        self.model.optimizer.step()
        self.states_history = []
        return loss

    def get_probs(self, state) :
        state = np.array(state).reshape(1,len(state))
        state = pad_sequences(state, maxlen = 29).reshape(1,29)
        guessed = np.array([1 if chr(i+ord('a')) in self.guessed else 0 for i in range(26)]).reshape(1,26)
        self.episode_memory.append((state,guessed))
        probs = self.model(state,guessed)
        probs = (torch.softmax(probs,dim=1)).squeeze()
        return probs

    def finalize_episode(self, answer) :
        inp_1, inp_2 = zip(*self.episode_memory)
        inp_1 = np.vstack(list(inp_1)).astype(float)      #stack the game state matrix
        inp_2 = np.vstack(list(inp_2)).astype(float)      #stack the one hot-encoded guessed matrix
        obj = 1.0 - inp_2                                 #compute the unused letters one-hot encoded
        len_ep = len(self.episode_memory)                 #length of episode
        correct_mask = np.array([[1 if chr(i+ord('a')) in answer else 0 for i in range(26)]]) # get mask from correct answer
        correct_mask = np.repeat(correct_mask, len_ep, axis = 0).astype(float)
        obj = correct_mask * obj  #the correct action is choosing the letters that are both unused AND exist in the word
        obj /= obj.sum(axis = 1).reshape(-1,1) #normalize so it sums to one
        self.states_history.append((inp_1, inp_2,obj))
        self.episode_memory = []
        self.guessed = set()

In [118]:
policy_net = Network()
player = NNAgent(policy_net)

save_episode = 500
view_episode = 100
update_episode = 5
avg_correct = 0
wins_avg = 0
n_trials = 20000
progbar = tqdm(range(n_trials))

""" 
Warmup Phase
Since the network can only collect data by interacting with the environment, 
to collect more data max_lives parameter or the maximum number of trial before game over is set 
slightly extra than the actual game parameter so episodes are longer. 
"""

game_params = {'max_tries' : 8}
env = Hangman(word_list, **game_params)

print('Training Start ...', end = '\n\n')

for episode_set in progbar :
    for _ in range(update_episode) :
        state = env.reset()
        done = False
        correct_count = 0
        while not done :
            guess = player.select_action(state)
            state, done, ans = env.step(guess)
        if env.correct_guesses == len(env.word):
            wins_avg += 1.0
        player.finalize_episode(ans['ans'])
        avg_correct += correct_count
    loss = player.train_model()
    progbar.set_description("Loss : {:.3f}              ".format(loss))

    if (episode_set +1) % view_episode == 0 :
        views = (episode_set + 1,avg_correct/(update_episode*view_episode), view_episode*update_episode, wins_avg/(update_episode*view_episode))
        print('Episode {} -------- Average Correct Count : {:.3f}     Last {} winrate : {:.3f}'.format(*views))
        if loss is not None :
            print('Loss :', loss)
            print()
            avg_correct = 0
            wins_avg = 0

    if (episode_set +1) % save_episode == 0 :
        player.model.save('model.pth')

print()
game_params = {'max_tries' : 6}
env = Hangman(word_list, **game_params)

n_trials = 70000
progbar = tqdm(range(n_trials))

for episode_set in progbar :
    for _ in range(update_episode) :
        total_reward = 0    
        state = env.reset()
        done = False
        correct_count = 0
        while not done :
            guess = player.select_action(state)
            state, done, ans = env.step(guess)
        if env.correct_guesses == len(env.word):
            wins_avg += 1.0
        player.finalize_episode(ans['ans'])
        avg_correct += correct_count
    loss = player.train_model()
    progbar.set_description("Loss : {:.3f}              ".format(loss))

    if (episode_set +1) % view_episode == 0 :
        views = (episode_set + 1,avg_correct/(update_episode*view_episode), view_episode, wins_avg/(update_episode*view_episode))
        print('Episode {} -------- Average Correct Count : {:.3f}     Last {} winrate : {:.3f}'.format(*views))
        if loss is not None :
            print('Loss :', loss)
            print()
            avg_correct = 0
            wins_avg = 0

    if (episode_set +1) % save_episode == 0 :
        player.model.save('model.pth')

  0%|          | 0/20000 [00:00<?, ?it/s]

Training Start ...



KeyboardInterrupt: 

In [87]:
guessed_letters = set()
policy = Network()
policy.load_state_dict(torch.load("model.pth"))
policy.eval()

def guess(self, word):
    len_word = len(word)
    state = np.array([ord(c)-ord('a')+1 if c != "_" else 27 for c in word])
    state = (pad_sequences([state],maxlen=29)).reshape(1,29)
    guessed = np.array([1 if chr(i+ord('a')) in self.guessed_letters else 0 for i in range(26)]).reshape(1,26)
    probs = torch.softmax(self.policy(state,guessed),dim=1).squeeze()
    i = 1
    sorted_probs = probs.argsort()
    while chr(sorted_probs[-i]+ord('a')) in self.guessed_letters :
        i+= 1
    idx = sorted_probs[-i]
    guess = chr(idx+ord('a'))
    guessed_letters.add(guess)
    return guess